# 0X Notebook-Template

In [1]:
import sys

sys.path.append("../../")

import pandas as pd
import connection as connsettings

from pyspark.sql import functions as f

from etl.etl import HadoopStdETL
from etl.clients import HiveClient, HDFileSystemClient
from etl.datamodels import HostDataClass, TableDataClass
from etl.enums import eHdfsFileType
from etl.connectors import CsvConnector

In [2]:
conn = CsvConnector(path="./database/datev.dbo.client.csv", sep="|")
conn

CsvConnector(
	'path=./database/datev.dbo.client.csv',
	'sep=|'
)

In [3]:
for i, batch in conn.iterbatches(batchsize=3):
    display(batch)

,id,name,address,change_field
0,57,Omicron Systems GmbH,"Musterstraße 57, 10163 Berlin",cf
1,19,Epic Solutions SE,"Musterstraße 19, 10125 Berlin",cf
2,66,Quantum Consulting SE,"Musterstraße 66, 10172 Berlin",cf


,id,name,address,change_field
3,30,Harmony Services GmbH,"Musterstraße 30, 10136 Berlin",cf
4,17,Echo Solutions SE,"Musterstraße 17, 10123 Berlin",cf
5,50,Mu Enterprises AG,"Musterstraße 50, 10156 Berlin",cf


,id,name,address,change_field
6,49,Momentum Innovations SE,"Musterstraße 49, 10155 Berlin",cf
7,92,Xcel Ventures AG,"Musterstraße 92, 10198 Berlin",cf
8,91,Wavelength Ventures AG,"Musterstraße 91, 10197 Berlin",cf


,id,name,address,change_field
9,3,Apex Corporation GmbH,"Musterstraße 3, 10109 Berlin",cf uc: 1


In [4]:
hive_host = HostDataClass(host=connsettings.HIVE_HOST, port=connsettings.HIVE_PORT)
hdfs_host = HostDataClass(host=connsettings.HDFS_HOST, port=connsettings.HDFS_PORT)
hdfs_client = HDFileSystemClient(hdfs_host=hdfs_host, hdfs_username="enricogoerlitz")
hive_client = HiveClient(host=hive_host, thrift_port=connsettings.HIVE_THRIFT_PORT)

In [5]:
hive_host = HostDataClass(host=connsettings.HIVE_HOST, port=connsettings.HIVE_PORT)
hdfs_host = HostDataClass(host=connsettings.HDFS_HOST, port=connsettings.HDFS_PORT)
hdfs_client = HDFileSystemClient(hdfs_host=hdfs_host, hdfs_username="enricogoerlitz")
hive_client = HiveClient(host=hive_host, thrift_port=connsettings.HIVE_THRIFT_PORT)

conn = CsvConnector(path="./database/datev.dbo.client.csv", sep="|")

etl = HadoopStdETL(
    conn=conn,
    hdfs_client=hdfs_client,
    hive_client=hive_client,
    tmp_path="/edw/hive/tmp/",
    bck_path="/edw/hive/bck_psa/",
    dist_path="/edw/hive/psa/",
    table=TableDataClass(
        database="datev",
        schema="dbo",
        table_name="client",
        pk=["id"]
    )
)

etl.start(batchsize=1)

/edw/hive/tmp/datev/dbo/client/BATCH0.parquet


ConnectTimeout: HTTPConnectionPool(host='hadoop-datanode-1', port=9864): Max retries exceeded with url: /webhdfs/v1/edw/hive/tmp/datev/dbo/client/BATCH0.parquet?op=CREATE&user.name=enricogoerlitz&namenoderpcaddress=192.168.64.102:50000&createflag=&createparent=true&overwrite=true&user.name=enricogoerlitz (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x1250e5810>, 'Connection to hadoop-datanode-1 timed out. (connect timeout=None)'))

In [ ]:
tmp_table = TableDataClass("tmp_client", "datev")
table = TableDataClass("client", "datev")

tmp_table, table

(TableDataClass(table_name='tmp_client', database='datev', schema=None, pk=None),
 TableDataClass(table_name='client', database='datev', schema=None, pk=None))

In [ ]:
hive_client.read_table(table).head()

,client.guid,client.pk,client.id,client.name,client.address,client.change_field,client.row_valid_to
0,7aee0b83-03ee-4ff3-b26e-52036c865af3,57,57,Omicron Systems GmbH,"Musterstraße 57, 10163 Berlin",cf,4133894400000000000
1,76336c71-9e27-4b88-a69d-93091d2b6563,19,19,Epic Solutions SE,"Musterstraße 19, 10125 Berlin",cf,4133894400000000000
2,4a9b5fc4-ec4e-4a21-9511-a85fc294bfd6,66,66,Quantum Consulting SE,"Musterstraße 66, 10172 Berlin",cf,4133894400000000000
3,62c05877-38dd-43d6-ac50-2e08cb9fdcd8,30,30,Harmony Services GmbH,"Musterstraße 30, 10136 Berlin",cf,4133894400000000000
4,d37ce9f4-0df3-483c-b814-3d81d1ca2392,17,17,Echo Solutions SE,"Musterstraße 17, 10123 Berlin",cf,4133894400000000000


In [ ]:
hive_client.read_table(tmp_table)

,tmp_client.guid,tmp_client.pk,tmp_client.id,tmp_client.name,tmp_client.address,tmp_client.change_field,tmp_client.row_valid_to


In [ ]:
for batch in hive_client.read_table_iterbatches(tmp_table, batchsize=3):
    display(batch)

In [ ]:
spark = hive_client.create_spark_session()
df_tmp_client = spark.sql(f"SELECT * FROM datev.tmp_client")
df_client = spark.sql(f"SELECT * FROM datev.client")
df_tmp_client.show()
df_client.show()

24/01/03 18:07:16 WARN Utils: Your hostname, MacBook-Air-von-Enrico.local resolves to a loopback address: 127.0.0.1; using 192.168.0.6 instead (on interface en0)
24/01/03 18:07:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/03 18:07:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/03 18:07:19 ERROR Executor: Exception in task 0.0 in stage 0.0 (TID 0)
org.apache.spark.SparkException: Parquet column cannot be converted in file hdfs://192.168.64.102:50000/edw/hive/tmp/datev/dbo/client/BATCH9.parquet. Column: [ROW_VALID_TO], Expected: string, Found: INT64.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:854)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextI

Py4JJavaError: An error occurred while calling o31.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (192.168.0.6 executor driver): org.apache.spark.SparkException: Parquet column cannot be converted in file hdfs://192.168.64.102:50000/edw/hive/tmp/datev/dbo/client/BATCH9.parquet. Column: [ROW_VALID_TO], Expected: string, Found: INT64.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:854)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:287)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:129)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:593)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBat

In [ ]:
df_tmp_client.join(df_client, on="pk").show()

+---+--------------------+---+--------------------+--------------------+------------+--------------+--------------------+---+--------------------+--------------------+------------+--------------+
| pk|                guid| id|                name|             address|change_field|row_is_current|                guid| id|                name|             address|change_field|row_is_current|
+---+--------------------+---+--------------------+--------------------+------------+--------------+--------------------+---+--------------------+--------------------+------------+--------------+
|  3|e0532323-a281-499...|  3|Apex Corporation ...|Musterstraße 3, 1...|    cf uc: 1|             1|86d8d699-1d00-41e...|  3|Apex Corporation ...|Musterstraße 3, 1...|    cf uc: 1|             1|
| 49|d74f63d8-c423-42a...| 49|Momentum Innovati...|Musterstraße 49, ...|          cf|             1|b5ec21ad-ac15-424...| 49|Momentum Innovati...|Musterstraße 49, ...|          cf|             1|
| 91|a7dd31f9-3d45-4

In [20]:
from datetime import datetime
df = pd.DataFrame({"id": [i for i in range(0, 10)]})
df["dt"] = pd.to_datetime(datetime(2010, 12, 31))
[i for i in df.dtypes.items()]

[('id', dtype('int64')), ('dt', dtype('<M8[ns]'))]

In [43]:
from pyspark.sql import functions as f

# update dummy
df_tmp_client = df_tmp_client.withColumn("name", f.when(f.col("pk") == 91, "here_name_changed").otherwise(f.col("name")))

# delete dummy
df_tmp_client = df_tmp_client.filter(f.col("pk") != 30)

df_client = df_client.withColumn("CHANGE_KEY", f.concat_ws("_", *["name", "change_field"]))
df_tmp_client = df_tmp_client.withColumn("CHANGE_KEY_TMP", f.concat_ws("_", *["name", "change_field"]))

df_joined = df_client.join(df_tmp_client, on="pk", how="left")


df_updated_rows = df_joined.filter(f.col("CHANGE_KEY") != f.col("CHANGE_KEY_TMP"))
df_deleted_rows = df_joined.filter(f.col("CHANGE_KEY_TMP").isNull())

#df_tmp_client.show()
# df_joined.show()
print("CHANGED COLUMNS:")
df_updated_rows.show()
df_deleted_rows.show()

CHANGED COLUMNS:
+---+--------------------+---+--------------------+--------------------+------------+--------------+--------------------+--------------------+---+-----------------+--------------------+------------+--------------+--------------------+
| pk|                guid| id|                name|             address|change_field|row_is_current|          CHANGE_KEY|                guid| id|             name|             address|change_field|row_is_current|      CHANGE_KEY_TMP|
+---+--------------------+---+--------------------+--------------------+------------+--------------+--------------------+--------------------+---+-----------------+--------------------+------------+--------------+--------------------+
| 91|cba0ecb1-1026-427...| 91|Wavelength Ventur...|Musterstraße 91, ...|          cf|             1|Wavelength Ventur...|a7dd31f9-3d45-483...| 91|here_name_changed|Musterstraße 91, ...|          cf|             1|here_name_changed_cf|
+---+--------------------+---+-------------

In [ ]:
hdfs_client.client.status("/edw/hive/tmp/")

{'accessTime': 0,
 'blockSize': 0,
 'childrenNum': 1,
 'fileId': 40344,
 'group': 'supergroup',
 'length': 0,
 'modificationTime': 1704281132017,
 'owner': 'enricogoerlitz',
 'pathSuffix': '',
 'permission': '755',
 'replication': 0,
 'storagePolicy': 0,
 'type': 'DIRECTORY'}